# Word sense disambiguation

In [4]:
!pip install nltk

In [5]:
import nltk
nltk.download('wordnet')
wn = nltk.corpus.wordnet
from nltk.wsd import lesk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import random
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import copy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.semi_supervised import LabelSpreading
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import ComplementNB

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data Preprocessing for baseline and Lesk's models

In [6]:
'''
@author: jcheung

Developed for Python 2. Automatically converted to Python 3; may result in bugs.
'''
import xml.etree.cElementTree as ET
import codecs

class WSDInstance:
    def __init__(self, my_id, lemma, context, index):
        self.id = my_id         # id of the WSD instance
        self.lemma = lemma      # lemma of the word whose sense is to be resolved
        self.context = context  # lemma of all the words in the sentential context
        self.index = index      # index of lemma within the context
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%s\t%s\t%s\t%d' % (self.id, self.lemma, b' '.join(self.context), self.index)

def load_instances(f):
    '''
    Load two lists of cases to perform WSD on. The structure that is returned is a dict, where
    the keys are the ids, and the values are instances of WSDInstance.
    '''
    tree = ET.parse(f)
    root = tree.getroot()

    dev_instances = {}
    test_instances = {}

    for text in root:
        if text.attrib['id'].startswith('d001'):
            instances = dev_instances
        else:
            instances = test_instances
        for sentence in text:
            # construct sentence context
            context = [to_ascii(el.attrib['lemma']) for el in sentence]
            for i, el in enumerate(sentence):
                if el.tag == 'instance':
                    my_id = el.attrib['id']
                    lemma = to_ascii(el.attrib['lemma'])
                    instances[my_id] = WSDInstance(my_id, lemma, context, i)
    return dev_instances, test_instances

def load_key(f):
    '''
    Load the solutions as dicts.
    Key is the id
    Value is the list of correct sense keys.
    '''
    dev_key = {}
    test_key = {}
    for line in open(f):
        if len(line) <= 1: continue
        #print (line)
        doc, my_id, sense_key = line.strip().split(' ', 2)
        if doc == 'd001':
            dev_key[my_id] = sense_key.split()
        else:
            test_key[my_id] = sense_key.split()
    return dev_key, test_key

def to_ascii(s):
    # remove all non-ascii characters
    return codecs.encode(s, 'ascii', 'ignore')

if __name__ == '__main__':
    data_f = 'multilingual-all-words.en.xml'
    key_f = 'wordnet.en.key'
    dev_instances, test_instances = load_instances(data_f)
    dev_key, test_key = load_key(key_f)

    # IMPORTANT: keys contain fewer entries than the instances; need to remove them
    dev_instances = {k:v for (k,v) in dev_instances.items() if k in dev_key}
    test_instances = {k:v for (k,v) in test_instances.items() if k in test_key}

    # read to use here
    print(len(dev_instances)) # number of dev instances
    print(len(test_instances)) # number of test instances


194
1450


In [7]:
# load the gold standards
with open('/content/wordnet.en.key') as f:
  lines = f.readlines()

In [8]:
# put the gold standard into a list
# each element is of the form (ID, word sense)
gold_standard = []
for line in lines:
    gold_standard.append(tuple(line.split(" ")[1:3]))

In [9]:
stop_words = stopwords.words('english')

In [10]:
# tokenize and lemmatize, remove stop words
# make a list of dev instances, each element is in the form
# (index, word, sentence)

dev_processed = []

for key in dev_instances.keys():
    # get the lemma
    word = dev_instances[key].lemma.decode('utf-8').lower()
    # get the sentence
    sentence = dev_instances[key].context
    # tokenize
    tokens = []
    for w in sentence:
        tokens.append(w.decode('utf-8').lower())
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    # remove stop words
    filtered_lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
    # get the index
    index = dev_instances[key].id
    # append to the list
    dev_processed.append((index, word, filtered_lemmas))

In [11]:
# tokenize and lemmatize, remove stop words
# make a list of dev instances, each element is in the form
# (index, word, sentence)

test_processed = []

for key in test_instances.keys():
    # get the lemma
    word = test_instances[key].lemma.decode('utf-8').lower()
    # get the sentence
    sentence = test_instances[key].context
    # tokenize
    tokens = []
    for w in sentence:
        tokens.append(w.decode('utf-8').lower())
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    # remove stop words
    filtered_lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
    # get the index
    index = test_instances[key].id
    # append to the list
    test_processed.append((index, word, filtered_lemmas))

## Baseline model: The most frequent sense

Go through all the test instances and assign them the most frequent sense.

In [12]:
# go through all the test cases and record the number of accurate word sense assignment
test_size = len(test_instances)
accurate_assignments = 0
for key in test_instances.keys():
    # get the lemma and the word sense
    word = test_instances[key].lemma.decode('utf-8').lower()
    sense_key = wn.synsets(word)[0].lemmas()[0].key()
    sense_key = sense_key.split("%")[1]

    # get the gold standard
    for line in lines:
        if key in line:
            gold_std = line.split(" ")[2]
            gold_std = gold_std.split("%")[1]

    if sense_key == gold_std:
        accurate_assignments += 1

In [13]:
# calculate the accuracy
accuracy = accurate_assignments / test_size
print("Accuracy for baseline model:", accuracy)
print()

Accuracy for baseline model: 0.5979310344827586



## NLTK's Lesk's algorithm model

Call NLTK's Lesk's algorithm model to perform word sense disambiguation.

In [14]:
# go through all the test cases and record the number of accurate word sense assignment
test_size = len(test_instances)
accurate_assignments = 0
for test_instance in test_processed:
    key = test_instance[0]

    # get the lemma and the word sense
    word = test_instance[1]
    sense_key = lesk(test_instance[2], word, 'n').lemmas()[0].key()
    sense_key = sense_key.split("%")[1]

    # get the gold standard
    for gs in gold_standard:
        if key == gs[0]:
            gold_std = gs[1]
            gold_std = gold_std.split("%")[1]

    if sense_key == gold_std:
        accurate_assignments += 1

In [15]:
# calculate the accuracy
accuracy = accurate_assignments / test_size
print("Accuracy for Lesk's model:", accuracy)
print()

Accuracy for Lesk's model: 0.3489655172413793



## Data for Semi-supervised and supervised ML models
Here, we choose 5 words to disambiguate. For each word, we build training and test datasets for it. We lemmatize and vectorize the words. Let's choose the 5 words that are in both dev set and test set and have the hightest occurances in the test set.

In [16]:
# words that are in both test and dev sets
overlap_words = []
for instance in test_processed:
    word = instance[1]
    for w in dev_processed:
        if word == w[1]:
            overlap_words.append(word)
            break

In [17]:
# Initialize an empty dictionary to store counts
word_counts = {}

# Count occurrences using a loop
for word in overlap_words:
    if word in word_counts:
        word_counts[word] += 1
    else:
        word_counts[word] = 1

In [18]:
# Get the 8 keys with the highest values
top_8_keys = sorted(word_counts, key=word_counts.get, reverse=True)[:8]

# Output the result
print("Words to disambiguate and their counts in the test set:")
for key in top_8_keys:
    print(f"{key}: {word_counts[key]}")
print()

Words to disambiguate and their counts in the test set:
game: 23
year: 22
country: 18
action: 11
claim: 10
time: 6
world: 6
issue: 6



In [19]:
# see the entries for the words in dev set
for word in top_8_keys:
    for entry in dev_processed:
        if word == entry[1]:
            print(entry)

('d001.s020.t001', 'game', ['clearly', 'game', 'new', 'economic', 'hegemony', 'develop', ',', 'say', 'ulate', ',', 'also', 'serve', 'regional', 'mexico', 'central_america', 'climate_change', 'adviser', 'conservation_international', '.'])
('d001.s002.t011', 'year', ['u.n.-sponsored', 'climate', 'conference', '--', 'characterize', 'far', 'unruly', 'posturing', 'mutual', 'recrimination', '--', 'gain', 'renewed', 'focus', 'friday', 'release', 'document', 'outline', 'ambitious', 'greenhouse-gas', 'reduction', 'next', '@card@', 'year', ',', 'industrialized_nation', 'shoulder', 'burden', 'near', 'term', '.'])
('d001.s011.t005', 'year', ['european_union', 'give', 'talk', 'boost', 'well', 'friday', 'pledge', 'provide', '$', '@card@', 'billion', 'year', 'next', 'three', 'year', 'help', 'poor', 'country', 'adapt', 'impact', 'climate_change', '--', 'cope', 'flood', 'drought', 'avoid', 'deforestation', '.'])
('d001.s011.t006', 'year', ['european_union', 'give', 'talk', 'boost', 'well', 'friday', 'p

In [20]:
# see the words in gold standard
for word in top_8_keys:
    for gs in gold_standard:
        if word == gs[1].split("%")[0]:
            print(gs)

('d001.s020.t001', 'game%1:09:00::')
('d002.s005.t001', 'game%1:04:03::')
('d002.s008.t003', 'game%1:04:03::')
('d002.s010.t001', 'game%1:04:03::')
('d002.s010.t007', 'game%1:04:03::')
('d002.s016.t004', 'game%1:04:03::')
('d007.s003.t006', 'game%1:04:03::')
('d007.s004.t003', 'game%1:04:03::')
('d007.s005.t007', 'game%1:04:03::')
('d007.s006.t004', 'game%1:04:03::')
('d007.s012.t009', 'game%1:04:03::')
('d007.s017.t004', 'game%1:04:03::')
('d007.s018.t002', 'game%1:04:03::')
('d007.s019.t008', 'game%1:04:03::')
('d007.s020.t011', 'game%1:04:03::')
('d011.s002.t003', 'game%1:04:03::')
('d011.s003.t004', 'game%1:04:03::')
('d011.s004.t005', 'game%1:04:03::')
('d011.s005.t011', 'game%1:04:03::')
('d011.s006.t003', 'game%1:04:03::')
('d011.s010.t007', 'game%1:04:03::')
('d011.s014.t003', 'game%1:04:03::')
('d011.s017.t001', 'game%1:04:03::')
('d011.s022.t001', 'game%1:04:03::')
('d001.s002.t011', 'year%1:28:01::')
('d001.s011.t005', 'year%1:28:01::')
('d001.s011.t006', 'year%1:28:01::')
(

In [21]:
# a function that reads the files, construct train and test sets,
# tokenize, lemmatize, remove stop words, and vectorize
# ngram can be "uni", "bi", "tri"

def output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, ngram):
    """
    return X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test
    """

    # opening the files
    with open(sense_1_file) as f:
        sense_1_corpus = f.read()

    with open(sense_2_file) as f:
        sense_2_corpus = f.read()

    with open(general_file) as f:
        unlabeled_corpus = f.read()
        unlabeled_corpus_list = unlabeled_corpus.split("\n")

    # attach the label to the corpus, shuffle it
    labeled_corpus_list = _

    sense_1_list = sense_1_corpus.split("\n")
    for i in range(len(sense_1_list)):
      sense_1_list[i] = (sense_1_list[i], 1)  # 1 means sense 1

    sense_2_list = sense_2_corpus.split("\n")
    for i in range(len(sense_2_list)):
      sense_2_list[i] = (sense_2_list[i], 2)  # 2 means sense 2

    # add the instances in the dev set
    ids_of_target_word = []

    for instance in dev_processed:
        if instance[1] == target_word:
            ids_of_target_word.append(instance[0])

    # retrieve the sentences
    for id in ids_of_target_word:
        # get the sentence
        sentence = ""
        for key in dev_instances.keys():
            if key == id:
                sentence = b' '.join(dev_instances[key].context).decode("utf-8")
        # get the gold standard
        for gs in gold_standard:
            if id == gs[0]:
                gold_sense = gs[1].split("%")[1]
                if gold_sense == sense_1:
                    sense_1_list.append((sentence, 1))
                if gold_sense == sense_2:
                    sense_2_list.append((sentence, 2))

    labeled_corpus_list = sense_1_list + sense_2_list
    random.Random(550).shuffle(labeled_corpus_list)
    print("Size of unlabeled training set: ", len(unlabeled_corpus_list))
    print("Size of labeled training set: ", len(labeled_corpus_list))

    # construct the test set for target word
    labeled_test_cases = []
    ids_of_target_word = []

    for instance in test_processed:
        if instance[1] == target_word:
            ids_of_target_word.append(instance[0])

    # retrieve the sentences
    for id in ids_of_target_word:
        # get the sentence
        sentence = ""
        for key in test_instances.keys():
            if key == id:
                sentence = b' '.join(test_instances[key].context).decode("utf-8")
        # get the gold standard
        for gs in gold_standard:
            if id == gs[0]:
                gold_sense = gs[1].split("%")[1]
                if gold_sense == sense_1:
                    labeled_test_cases.append((sentence, 1))
                if gold_sense == sense_2:
                    labeled_test_cases.append((sentence, 2))

    print("Size of test set (labeled): ", len(labeled_test_cases))

    # develop training and test set

    X_train_labeled = []
    y_train_labeled = []
    for i in range(len(labeled_corpus_list)):
        X_train_labeled.append(labeled_corpus_list[i][0])
        y_train_labeled.append(labeled_corpus_list[i][1])

    X_train_semi = []
    y_train_semi = []

    for i in range(len(unlabeled_corpus_list)):
        X_train_semi.append(unlabeled_corpus_list[i])
        y_train_semi.append(-1)  # -1 means unlabeled

    X_train_semi_l = copy.deepcopy(X_train_labeled)
    X_train_semi += X_train_semi_l
    y_train_semi += y_train_labeled

    X_test = []
    y_test = []
    for i in range(len(labeled_test_cases)):
        X_test.append(labeled_test_cases[i][0])
        y_test.append(labeled_test_cases[i][1])

    # tokenize the words in each datapoint and then lemmatize the words

    wordnet_lemmatizer = WordNetLemmatizer()

    for i in range(len(X_train_semi)):
        X_train_semi[i] = X_train_semi[i].lower()
        X_train_semi[i]= X_train_semi[i].replace(",", "").replace(".", "").replace("'", "").replace("-", " ").replace("@", "")
        X_train_semi[i] = word_tokenize(X_train_semi[i])

        for j in range(len(X_train_semi[i])):
          X_train_semi[i][j] = wordnet_lemmatizer.lemmatize(X_train_semi[i][j])

        X_train_semi[i] = " ".join(X_train_semi[i])

    for i in range(len(X_train_labeled)):
        X_train_labeled[i] = X_train_labeled[i].lower()
        X_train_labeled[i]= X_train_labeled[i].replace(",", "").replace(".", "").replace("'", "").replace("-", " ").replace("@", "")
        X_train_labeled[i] = word_tokenize(X_train_labeled[i])

        for j in range(len(X_train_labeled[i])):
          X_train_labeled[i][j] = wordnet_lemmatizer.lemmatize(X_train_labeled[i][j])

        X_train_labeled[i] = " ".join(X_train_labeled[i])

    # count-vectorize the words
    # Create a Vectorizer Object
    if ngram == "uni":
        vectorizer = CountVectorizer(ngram_range = (1, 1), stop_words='english')
    elif ngram == "bi":
        vectorizer = CountVectorizer(ngram_range = (2, 2), stop_words='english')
    elif ngram == "tri":
        vectorizer = CountVectorizer(ngram_range = (3, 3), stop_words='english')

    vectorizer.fit(X_train_semi)

    # Encode X_train_semi and X_train_labeled
    X_train_semi = vectorizer.transform(X_train_semi).toarray()
    X_train_labeled = vectorizer.transform(X_train_labeled).toarray()

    # Define a function to streamline lemmatization and vectorization
    def lemma_preprocess(list_of_sen):
        for i in range(len(list_of_sen)):
            list_of_sen[i] = list_of_sen[i].lower()
            list_of_sen[i]= list_of_sen[i].replace(",", "").replace(".", "").replace("'", "").replace("-", " ")
            list_of_sen[i] = word_tokenize(list_of_sen[i])

            for j in range(len(list_of_sen[i])):
                list_of_sen[i][j] = wordnet_lemmatizer.lemmatize(list_of_sen[i][j])

            list_of_sen[i] = " ".join(list_of_sen[i])

        return vectorizer.transform(list_of_sen).toarray()

    # Lemmatize and vectorize the test data
    X_test = lemma_preprocess(X_test)

    return X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test

## Semi-supervised/Boostrapping model


### Word: country

We pick two senses of country. They are:
1.  S: (n) state (state%1:14:00::), nation (nation%1:14:00::), country (country%1:14:00::), land (land%1:14:00::), commonwealth (commonwealth%1:14:00::), res publica (res_publica%1:14:00::), body politic (body_politic%1:14:00::) (a politically organized body of people under a single government) "the state has elected a new president"; "African nations"; "students who had come to the nation's capitol"; "the country's largest manufacturer"; "an industrialized land".
2.  S: (n) country (country%1:15:01::), rural area (rural_area%1:15:00::) (an area outside of cities and towns) "his poetry celebrated the slower pace of life in the country"

We assign 1 to the first sense and 2 to the second sense.

In [22]:
# parameters to change
target_word = "country"
sense_1_file = "/content/country_nation.txt"
sense_2_file = "/content/country_rural.txt"
general_file = "/content/country.txt"
sense_1 = "1:14:00::"
sense_2 = "1:15:01::"

Let's try out unigram, bigram, and trigram models.

In [23]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  107
Size of test set (labeled):  18


In [24]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [3.0, 5.0, 10.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.001, 0.01, 0.05, 0.1],
              'max_iter': [1, 2, 3, 5]}

semi_uni_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_uni_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.05, 0.1],
                         'gamma': [3.0, 5.0, 10.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5],
                         'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [25]:
print("Semi-Supervised Model \"country\" with Unigram Proprocessing - Best Hyperparameters\n")
print(semi_uni_model.best_params_)
print()

Semi-Supervised Model "country" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 0.001, 'gamma': 5.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [26]:
semi_uni_model_predictions = semi_uni_model.predict(X_test)
semi_uni_report = classification_report(y_test, semi_uni_model_predictions)
print("Semi-Supervised Model \"country\" with Unigram Proprocessing - Performance Report\n")
print(semi_uni_report)
print()

Semi-Supervised Model "country" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.94      0.97        18
           2       0.00      0.00      0.00         0

    accuracy                           0.94        18
   macro avg       0.50      0.47      0.49        18
weighted avg       1.00      0.94      0.97        18




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print(semi_uni_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  107
Size of test set (labeled):  18


In [29]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [2.0, 3.0, 4.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.001, 0.01, 0.05, 0.1],
              'max_iter': [1, 2, 3, 5]}

semi_bi_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_bi_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.05, 0.1],
                         'gamma': [2.0, 3.0, 4.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5],
                         'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [30]:
print("Semi-Supervised Model \"country\" with Bigram Proprocessing - Best Hyperparameters\n")
print(semi_bi_model.best_params_)
print()

Semi-Supervised Model "country" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 0.001, 'gamma': 3.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [31]:
semi_bi_model_predictions = semi_bi_model.predict(X_test)
semi_bi_report = classification_report(y_test, semi_bi_model_predictions)
print("Semi-Supervised Model \"country\" with Bigram Proprocessing - Performance Report\n")
print(semi_bi_report)
print()

Semi-Supervised Model "country" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18




In [32]:
print(semi_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  107
Size of test set (labeled):  18


In [34]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [1.0, 2.0, 3.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.00001, 0.0001, .001, 0.01],
              'max_iter': [1, 2, 3]}

semi_tri_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_tri_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01],
                         'gamma': [1.0, 2.0, 3.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3], 'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [35]:
print("Semi-Supervised Model \"country\" with Trigram Proprocessing - Best Hyperparameters\n")
print(semi_tri_model.best_params_)
print()

Semi-Supervised Model "country" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-05, 'gamma': 1.0, 'kernel': 'knn', 'max_iter': 1, 'n_neighbors': 1}



In [36]:
semi_tri_model_predictions = semi_tri_model.predict(X_test)
semi_tri_report = classification_report(y_test, semi_tri_model_predictions)
print("Semi-Supervised Model \"country\" with Trigram Proprocessing - Performance Report\n")
print(semi_tri_report)
print()

Semi-Supervised Model "country" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18




/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [37]:
print(semi_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Word: game

We pick two senses of game. They are:
1.  S: (n) plot (plot%1:09:00::), secret plan (secret_plan%1:09:00::), game (game%1:09:00::) (a secret scheme to do something (especially something underhand or illegal)) "they concocted a plot to discredit the governor"; "I saw through his little game from the start"
2.  S: (n) game (game%1:04:03::) (a single play of a sport or other contest) "the game lasted two hours"

We assign 1 to the first sense and 2 to the second sense.

In [38]:
# parameters to change
target_word = "game"
sense_1_file = "/content/game_secret.txt"
sense_2_file = "/content/game_contest.txt"
general_file = "/content/game.txt"
sense_1 = "1:09:00::"
sense_2 = "1:04:03::"

In [39]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  23


In [40]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [8.0, 10.0, 12.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.001, 0.01, 0.05, 0.1],
              'max_iter': [1, 2, 3, 5]}

semi_uni_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_uni_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=2 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.05, 0.1],
                         'gamma': [8.0, 10.0, 12.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5],
                         'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [41]:
print("Semi-Supervised Model \"game\" with Unigram Proprocessing - Best Hyperparameters\n")
print(semi_uni_model.best_params_)
print()

Semi-Supervised Model "game" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 0.01, 'gamma': 10.0, 'kernel': 'rbf', 'max_iter': 2, 'n_neighbors': 1}



In [42]:
semi_uni_model_predictions = semi_uni_model.predict(X_test)
semi_uni_report = classification_report(y_test, semi_uni_model_predictions)
print("Semi-Supervised Model \"game\" with Unigram Proprocessing - Performance Report\n")
print(semi_uni_report)
print()

Semi-Supervised Model "game" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.74      0.85        23

    accuracy                           0.74        23
   macro avg       0.50      0.37      0.42        23
weighted avg       1.00      0.74      0.85        23




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
print(semi_uni_model_predictions)
print(y_test)

[1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 1 2 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [44]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  23


In [45]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [2.0, 3.0, 4.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.0001, 0.001, 0.01],
              'max_iter': [1, 2, 3, 5]}

semi_bi_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_bi_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01],
                         'gamma': [2.0, 3.0, 4.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5],
                         'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [46]:
print("Semi-Supervised Model \"game\" with Bigram Proprocessing - Best Hyperparameters\n")
print(semi_bi_model.best_params_)
print()

Semi-Supervised Model "game" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 0.0001, 'gamma': 3.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [47]:
semi_bi_model_predictions = semi_bi_model.predict(X_test)
semi_bi_report = classification_report(y_test, semi_bi_model_predictions)
print("Semi-Supervised Model \"game\" with Bigram Proprocessing - Performance Report\n")
print(semi_bi_report)
print()

Semi-Supervised Model "game" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.09      0.16        23

    accuracy                           0.09        23
   macro avg       0.50      0.04      0.08        23
weighted avg       1.00      0.09      0.16        23




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
print(semi_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [49]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  23


In [50]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [1.0, 2.0, 3.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.00001, 0.0001, .001, 0.01],
              'max_iter': [1, 2, 3]}

semi_tri_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_tri_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01],
                         'gamma': [1.0, 2.0, 3.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3], 'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [51]:
print("Semi-Supervised Model \"game\" with Trigram Proprocessing - Best Hyperparameters\n")
print(semi_tri_model.best_params_)
print()

Semi-Supervised Model "game" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-05, 'gamma': 1.0, 'kernel': 'knn', 'max_iter': 1, 'n_neighbors': 1}



In [52]:
semi_tri_model_predictions = semi_tri_model.predict(X_test)
semi_tri_report = classification_report(y_test, semi_tri_model_predictions)
print("Semi-Supervised Model \"game\" with Trigram Proprocessing - Performance Report\n")
print(semi_tri_report)
print()

Semi-Supervised Model "game" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00      23.0

    accuracy                           0.00      23.0
   macro avg       0.00      0.00      0.00      23.0
weighted avg       0.00      0.00      0.00      23.0




/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [53]:
print(semi_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### Word: action

We pick two senses of action. They are:
1.  S: (n) action (action%1:04:02::) (something done (usually as opposed to something said)) "there were stories of murders and other unnatural actions"
2.  S: (n) action (action%1:04:04::) (an act by a government body or supranational organization) "recent federal action undermined the segregationist position"; "the United Nations must have the power to propose and organize action without being hobbled by irrelevant issues"; "the Union action of emancipating Southern slaves"

We assign 1 to the first sense and 2 to the second sense.

In [54]:
# parameters to change
target_word = "action"
sense_1_file = "/content/action_done.txt"
sense_2_file = "/content/action_gov.txt"
general_file = "/content/action.txt"
sense_1 = "1:04:02::"
sense_2 = "1:04:04::"

In [55]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  104
Size of test set (labeled):  11


In [56]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [8.0, 10.0, 12.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.001, 0.01, 0.05, 0.1],
              'max_iter': [1, 2, 3, 5]}

semi_uni_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_uni_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=2 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.05, 0.1],
                         'gamma': [8.0, 10.0, 12.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5],
                         'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [57]:
print("Semi-Supervised Model \"action\" with Unigram Proprocessing - Best Hyperparameters\n")
print(semi_uni_model.best_params_)
print()

Semi-Supervised Model "action" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 0.01, 'gamma': 10.0, 'kernel': 'rbf', 'max_iter': 2, 'n_neighbors': 1}



In [58]:
semi_uni_model_predictions = semi_uni_model.predict(X_test)
semi_uni_report = classification_report(y_test, semi_uni_model_predictions)
print("Semi-Supervised Model \"action\" with Unigram Proprocessing - Performance Report\n")
print(semi_uni_report)
print()

Semi-Supervised Model "action" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        11

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11




In [59]:
print(semi_uni_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [60]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  104
Size of test set (labeled):  11


In [61]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [3.0, 4.0, 5.0],
              'n_neighbors': [1, 2, 3],
              'alpha': [0.0001, 0.001, 0.01],
              'max_iter': [1, 2, 3, 5]}

semi_bi_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_bi_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01],
                         'gamma': [3.0, 4.0, 5.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5], 'n_neighbors': [1, 2, 3]},
             verbose=3)

In [62]:
print("Semi-Supervised Model \"action\" with Bigram Proprocessing - Best Hyperparameters\n")
print(semi_bi_model.best_params_)
print()

Semi-Supervised Model "action" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 0.0001, 'gamma': 4.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [63]:
semi_bi_model_predictions = semi_bi_model.predict(X_test)
semi_bi_report = classification_report(y_test, semi_bi_model_predictions)
print("Semi-Supervised Model \"action\" with Bigram Proprocessing - Performance Report\n")
print(semi_bi_report)
print()

Semi-Supervised Model "action" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        11

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11




In [64]:
print(semi_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [65]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  104
Size of test set (labeled):  11


In [66]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [3.0, 4.0, 5.0],
              'n_neighbors': [1, 2, 3],
              'alpha': [0.00001, 0.0001, .001, 0.01],
              'max_iter': [1, 2, 3]}

semi_tri_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_tri_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01],
                         'gamma': [3.0, 4.0, 5.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3], 'n_neighbors': [1, 2, 3]},
             verbose=3)

In [67]:
print("Semi-Supervised Model \"action\" with Trigram Proprocessing - Best Hyperparameters\n")
print(semi_tri_model.best_params_)
print()

Semi-Supervised Model "action" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-05, 'gamma': 4.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [68]:
semi_tri_model_predictions = semi_tri_model.predict(X_test)
semi_tri_report = classification_report(y_test, semi_tri_model_predictions)
print("Semi-Supervised Model \"action\" with Trigram Proprocessing - Performance Report\n")
print(semi_tri_report)
print()

Semi-Supervised Model "action" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        11

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11




In [69]:
print(semi_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Word: claim

We pick two senses of claim. They are:
1.  S: (n) claim (claim%1:10:02::) (an assertion that something is true or factual) "his claim that he was innocent"; "evidence contradicted the government's claims"
2.  S: (n) claim (claim%1:04:00::) (demand for something as rightful or due) "they struck in support of their claim for a shorter work day"

We assign 1 to the first sense and 2 to the second sense.

In [70]:
# parameters to change
target_word = "claim"
sense_1_file = "/content/claim_assertion.txt"
sense_2_file = "/content/claim_demand.txt"
general_file = "/content/claim.txt"
sense_1 = "1:10:02::"
sense_2 = "1:04:00::"

In [71]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  9


In [72]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [10.0, 12.0, 14.0],
              'n_neighbors': [1, 2, 3, 4],
              'alpha': [0.00001, 0.0001, 0.001],
              'max_iter': [1, 2, 3, 5]}

semi_uni_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_uni_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=2 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001],
                         'gamma': [10.0, 12.0, 14.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3, 5],
                         'n_neighbors': [1, 2, 3, 4]},
             verbose=3)

In [73]:
print("Semi-Supervised Model \"claim\" with Unigram Proprocessing - Best Hyperparameters\n")
print(semi_uni_model.best_params_)
print()

Semi-Supervised Model "claim" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 0.0001, 'gamma': 12.0, 'kernel': 'rbf', 'max_iter': 2, 'n_neighbors': 1}



In [74]:
semi_uni_model_predictions = semi_uni_model.predict(X_test)
semi_uni_report = classification_report(y_test, semi_uni_model_predictions)
print("Semi-Supervised Model \"claim\" with Unigram Proprocessing - Performance Report\n")
print(semi_uni_report)
print()

Semi-Supervised Model "claim" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.89      0.94         9
           2       0.00      0.00      0.00         0

    accuracy                           0.89         9
   macro avg       0.50      0.44      0.47         9
weighted avg       1.00      0.89      0.94         9




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
print(semi_uni_model_predictions)
print(y_test)

[1 1 1 1 1 1 2 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [76]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  9


In [77]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [1.0, 2.0, 3.0, 4.0],
              'n_neighbors': [1, 2],
              'alpha': [0.00001, 0.0001, 0.001],
              'max_iter': [1, 2, 3]}

semi_bi_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_bi_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001],
                         'gamma': [1.0, 2.0, 3.0, 4.0],
                         'kernel': ['knn', 'rbf'], 'max_iter': [1, 2, 3],
                         'n_neighbors': [1, 2]},
             verbose=3)

In [78]:
print("Semi-Supervised Model \"claim\" with Bigram Proprocessing - Best Hyperparameters\n")
print(semi_bi_model.best_params_)
print()

Semi-Supervised Model "claim" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-05, 'gamma': 2.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [79]:
semi_bi_model_predictions = semi_bi_model.predict(X_test)
semi_bi_report = classification_report(y_test, semi_bi_model_predictions)
print("Semi-Supervised Model \"claim\" with Bigram Proprocessing - Performance Report\n")
print(semi_bi_report)
print()

Semi-Supervised Model "claim" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.89      0.94         9
           2       0.00      0.00      0.00         0

    accuracy                           0.89         9
   macro avg       0.50      0.44      0.47         9
weighted avg       1.00      0.89      0.94         9




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
print(semi_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 2 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [81]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  9


In [82]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [1.0, 2.0, 3.0, 4.0],
              'n_neighbors': [1, 2],
              'alpha': [0.00001, 0.0001, 0.001],
              'max_iter': [1, 2, 3]}

semi_tri_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_tri_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001],
                         'gamma': [1.0, 2.0, 3.0, 4.0],
                         'kernel': ['knn', 'rbf'], 'max_iter': [1, 2, 3],
                         'n_neighbors': [1, 2]},
             verbose=3)

In [83]:
print("Semi-Supervised Model \"claim\" with Trigram Proprocessing - Best Hyperparameters\n")
print(semi_tri_model.best_params_)
print()

Semi-Supervised Model "claim" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-05, 'gamma': 2.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [84]:
semi_tri_model_predictions = semi_tri_model.predict(X_test)
semi_tri_report = classification_report(y_test, semi_tri_model_predictions)
print("Semi-Supervised Model \"claim\" with Trigram Proprocessing - Performance Report\n")
print(semi_tri_report)
print()

Semi-Supervised Model "claim" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9




In [85]:
print(semi_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]


### Word: time

We pick two senses of time. They are:
1.  S: (n) time (time%1:11:00::), clip (clip%1:11:00::) (an instance or single occasion for some event) "this time he succeeded"; "he called four times"; "he could do ten at a clip"
2. S: (n) time (time%1:28:05::) (a period of time considered as a resource under your control and sufficient to accomplish something) "take time to smell the roses"; "I didn't have time to finish"; "it took more than half my time"; "he waited for a long time"

We assign 1 to the first sense and 2 to the second sense.

In [86]:
# parameters to change
target_word = "time"
sense_1_file = "/content/time_event.txt"
sense_2_file = "/content/time_resource.txt"
general_file = "/content/time.txt"
sense_1 = "1:11:00::"
sense_2 = "1:28:05::"

In [87]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  6


In [88]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [8.0, 10.0, 12.0],
              'n_neighbors': [1, 2, 3],
              'alpha': [0.00001, 0.0001, 0.001],
              'max_iter': [1, 2, 3]}

semi_uni_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_uni_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=2 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001],
                         'gamma': [8.0, 10.0, 12.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3], 'n_neighbors': [1, 2, 3]},
             verbose=3)

In [89]:
print("Semi-Supervised Model \"time\" with Unigram Proprocessing - Best Hyperparameters\n")
print(semi_uni_model.best_params_)
print()

Semi-Supervised Model "time" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 0.0001, 'gamma': 10.0, 'kernel': 'rbf', 'max_iter': 2, 'n_neighbors': 1}



In [90]:
semi_uni_model_predictions = semi_uni_model.predict(X_test)
semi_uni_report = classification_report(y_test, semi_uni_model_predictions)
print("Semi-Supervised Model \"time\" with Unigram Proprocessing - Performance Report\n")
print(semi_uni_report)
print()

Semi-Supervised Model "time" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.50      0.67         4
           2       0.50      1.00      0.67         2

    accuracy                           0.67         6
   macro avg       0.75      0.75      0.67         6
weighted avg       0.83      0.67      0.67         6




In [91]:
print(semi_uni_model_predictions)
print(y_test)

[1 2 2 2 2 1]
[1, 1, 2, 2, 1, 1]


In [92]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  6


In [93]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [6.0, 8.0, 10.0],
              'n_neighbors': [1, 2, 3],
              'alpha': [0.00001, 0.0001, 0.001],
              'max_iter': [1, 2, 3]}

semi_bi_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_bi_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001],
                         'gamma': [6.0, 8.0, 10.0], 'kernel': ['knn', 'rbf'],
                         'max_iter': [1, 2, 3], 'n_neighbors': [1, 2, 3]},
             verbose=3)

In [94]:
print("Semi-Supervised Model \"time\" with Bigram Proprocessing - Best Hyperparameters\n")
print(semi_bi_model.best_params_)
print()

Semi-Supervised Model "time" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-05, 'gamma': 6.0, 'kernel': 'rbf', 'max_iter': 1, 'n_neighbors': 1}



In [95]:
semi_bi_model_predictions = semi_bi_model.predict(X_test)
semi_bi_report = classification_report(y_test, semi_bi_model_predictions)
print("Semi-Supervised Model \"time\" with Bigram Proprocessing - Performance Report\n")
print(semi_bi_report)
print()

Semi-Supervised Model "time" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.33      1.00      0.50         2

    accuracy                           0.33         6
   macro avg       0.17      0.50      0.25         6
weighted avg       0.11      0.33      0.17         6




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [96]:
print(semi_bi_model_predictions)
print(y_test)

[2 2 2 2 2 2]
[1, 1, 2, 2, 1, 1]


In [97]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  6


In [98]:
param_grid = {'kernel': ['knn', 'rbf'],
              'gamma': [1.0, 2.0],
              'n_neighbors': [1, 2, 3],
              'alpha': [0.0000001, 0.000001, 0.00001],
              'max_iter': [1, 2, 3]}

semi_tri_model = GridSearchCV(LabelSpreading(), param_grid, refit=True, verbose=3, n_jobs=-1)
semi_tri_model.fit(X_train_semi, y_train_semi)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1 was reached without convergence.
  warnings.warn(


GridSearchCV(estimator=LabelSpreading(), n_jobs=-1,
             param_grid={'alpha': [1e-07, 1e-06, 1e-05], 'gamma': [1.0, 2.0],
                         'kernel': ['knn', 'rbf'], 'max_iter': [1, 2, 3],
                         'n_neighbors': [1, 2, 3]},
             verbose=3)

In [99]:
print("Semi-Supervised Model \"time\" with Trigram Proprocessing - Best Hyperparameters\n")
print(semi_tri_model.best_params_)
print()

Semi-Supervised Model "time" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-07, 'gamma': 1.0, 'kernel': 'knn', 'max_iter': 1, 'n_neighbors': 2}



In [100]:
semi_tri_model_predictions = semi_tri_model.predict(X_test)
semi_tri_report = classification_report(y_test, semi_tri_model_predictions)
print("Semi-Supervised Model \"time\" with Trigram Proprocessing - Performance Report\n")
print(semi_tri_report)
print()

Semi-Supervised Model "time" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.67      1.00      0.80         4
           2       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6




/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Us

In [101]:
print(semi_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1]
[1, 1, 2, 2, 1, 1]


## Naive Bayes model

### Word: country

We pick two senses of country. They are:
1.  S: (n) state (state%1:14:00::), nation (nation%1:14:00::), country (country%1:14:00::), land (land%1:14:00::), commonwealth (commonwealth%1:14:00::), res publica (res_publica%1:14:00::), body politic (body_politic%1:14:00::) (a politically organized body of people under a single government) "the state has elected a new president"; "African nations"; "students who had come to the nation's capitol"; "the country's largest manufacturer"; "an industrialized land".
2.  S: (n) country (country%1:15:01::), rural area (rural_area%1:15:00::) (an area outside of cities and towns) "his poetry celebrated the slower pace of life in the country"

We assign 1 to the first sense and 2 to the second sense.

In [102]:
# parameters to change
target_word = "country"
sense_1_file = "/content/country_nation.txt"
sense_2_file = "/content/country_rural.txt"
general_file = "/content/country.txt"
sense_1 = "1:14:00::"
sense_2 = "1:15:01::"

Let's try out unigram, bigram, and trigram models.

In [103]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  107
Size of test set (labeled):  18


In [104]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_uni_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_uni_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [105]:
print("Complement Naive Bayes Model \"country\" with Unigram Proprocessing - Best Hyperparameters\n")
print(cnb_uni_model.best_params_)
print()

Complement Naive Bayes Model "country" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [106]:
cnb_uni_model_predictions = cnb_uni_model.predict(X_test)
cnb_uni_report = classification_report(y_test, cnb_uni_model_predictions)
print("Complement Naive Bayes Model \"country\" with Unigram Proprocessing - Performance Report\n")
print(cnb_uni_report)
print()

Complement Naive Bayes Model "country" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.89      0.94        18
           2       0.00      0.00      0.00         0

    accuracy                           0.89        18
   macro avg       0.50      0.44      0.47        18
weighted avg       1.00      0.89      0.94        18




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
print(cnb_uni_model_predictions)
print(y_test)

[1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [108]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  107
Size of test set (labeled):  18


In [109]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_bi_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_bi_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [110]:
print("Complement Naive Bayes Model \"country\" with Bigram Proprocessing - Best Hyperparameters\n")
print(cnb_bi_model.best_params_)
print()

Complement Naive Bayes Model "country" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [111]:
cnb_bi_model_predictions = cnb_bi_model.predict(X_test)
cnb_bi_report = classification_report(y_test, cnb_bi_model_predictions)
print("Complement Naive Bayes Model \"country\" with Bigram Proprocessing - Performance Report\n")
print(cnb_bi_report)
print()

Complement Naive Bayes Model "country" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18




In [112]:
print(cnb_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [113]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  107
Size of test set (labeled):  18


In [114]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_tri_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_tri_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:1052: RuntimeWarning: divide by zero encountered in log
  logged = np.log(comp_count / comp_count.sum(axis=1, keepdims=True))


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [115]:
print("Complement Naive Bayes Model \"country\" with Trigram Proprocessing - Best Hyperparameters\n")
print(cnb_tri_model.best_params_)
print()

Complement Naive Bayes Model "country" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 0, 'class_prior': [0.5, 0.5]}



In [116]:
cnb_tri_model_predictions = cnb_tri_model.predict(X_test)
cnb_tri_report = classification_report(y_test, cnb_tri_model_predictions)
print("Complement Naive Bayes Model \"country\" with Trigram Proprocessing - Performance Report\n")
print(cnb_tri_report)
print()

Complement Naive Bayes Model "country" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18




/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [117]:
print(cnb_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Word: game

We pick two senses of game. They are:
1.  S: (n) plot (plot%1:09:00::), secret plan (secret_plan%1:09:00::), game (game%1:09:00::) (a secret scheme to do something (especially something underhand or illegal)) "they concocted a plot to discredit the governor"; "I saw through his little game from the start"
2.  S: (n) game (game%1:04:03::) (a single play of a sport or other contest) "the game lasted two hours"

We assign 1 to the first sense and 2 to the second sense.

In [118]:
# parameters to change
target_word = "game"
sense_1_file = "/content/game_secret.txt"
sense_2_file = "/content/game_contest.txt"
general_file = "/content/game.txt"
sense_1 = "1:09:00::"
sense_2 = "1:04:03::"

In [119]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  23


In [120]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_uni_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_uni_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [121]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Best Hyperparameters\n")
print(cnb_uni_model.best_params_)
print()

Complement Naive Bayes Model "game" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [122]:
cnb_uni_model_predictions = cnb_uni_model.predict(X_test)
cnb_uni_report = classification_report(y_test, cnb_uni_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Performance Report\n")
print(cnb_uni_report)
print()

Complement Naive Bayes Model "game" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.91      0.95        23

    accuracy                           0.91        23
   macro avg       0.50      0.46      0.48        23
weighted avg       1.00      0.91      0.95        23




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [123]:
print(cnb_uni_model_predictions)
print(y_test)

[1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [124]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  23


In [125]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_bi_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_bi_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [126]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Best Hyperparameters\n")
print(cnb_bi_model.best_params_)
print()

Complement Naive Bayes Model "game" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [127]:
cnb_bi_model_predictions = cnb_bi_model.predict(X_test)
cnb_bi_report = classification_report(y_test, cnb_bi_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Performance Report\n")
print(cnb_bi_report)
print()

Complement Naive Bayes Model "game" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.26      0.41        23

    accuracy                           0.26        23
   macro avg       0.50      0.13      0.21        23
weighted avg       1.00      0.26      0.41        23




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [128]:
print(cnb_bi_model_predictions)
print(y_test)

[1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 2 2 1 1 1 2 2 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [129]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  23


In [130]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_tri_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_tri_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [131]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Best Hyperparameters\n")
print(cnb_tri_model.best_params_)
print()

Complement Naive Bayes Model "game" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [132]:
cnb_tri_model_predictions = cnb_tri_model.predict(X_test)
cnb_tri_report = classification_report(y_test, cnb_tri_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Performance Report\n")
print(cnb_tri_report)
print()

Complement Naive Bayes Model "game" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00      23.0

    accuracy                           0.00      23.0
   macro avg       0.00      0.00      0.00      23.0
weighted avg       0.00      0.00      0.00      23.0




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [133]:
print(cnb_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### Word: action

We pick two senses of action. They are:
1.  S: (n) action (action%1:04:02::) (something done (usually as opposed to something said)) "there were stories of murders and other unnatural actions"
2.  S: (n) action (action%1:04:04::) (an act by a government body or supranational organization) "recent federal action undermined the segregationist position"; "the United Nations must have the power to propose and organize action without being hobbled by irrelevant issues"; "the Union action of emancipating Southern slaves"

We assign 1 to the first sense and 2 to the second sense.

In [134]:
# parameters to change
target_word = "action"
sense_1_file = "/content/action_done.txt"
sense_2_file = "/content/action_gov.txt"
general_file = "/content/action.txt"
sense_1 = "1:04:02::"
sense_2 = "1:04:04::"

In [135]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  104
Size of test set (labeled):  11


In [136]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_uni_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_uni_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [137]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Best Hyperparameters\n")
print(cnb_uni_model.best_params_)
print()

Complement Naive Bayes Model "action" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [138]:
cnb_uni_model_predictions = cnb_uni_model.predict(X_test)
cnb_uni_report = classification_report(y_test, cnb_uni_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Performance Report\n")
print(cnb_uni_report)
print()

Complement Naive Bayes Model "action" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.55      0.71        11
           2       0.00      0.00      0.00         0

    accuracy                           0.55        11
   macro avg       0.50      0.27      0.35        11
weighted avg       1.00      0.55      0.71        11




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [139]:
print(cnb_uni_model_predictions)
print(y_test)

[2 2 1 2 2 1 1 2 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [140]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  104
Size of test set (labeled):  11


In [141]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_bi_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_bi_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [142]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Best Hyperparameters\n")
print(cnb_bi_model.best_params_)
print()

Complement Naive Bayes Model "action" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [143]:
cnb_bi_model_predictions = cnb_bi_model.predict(X_test)
cnb_bi_report = classification_report(y_test, cnb_bi_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Performance Report\n")
print(cnb_bi_report)
print()

Complement Naive Bayes Model "action" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        11

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11




In [144]:
print(cnb_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [145]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  104
Size of test set (labeled):  11


In [146]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_tri_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_tri_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [147]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Best Hyperparameters\n")
print(cnb_tri_model.best_params_)
print()

Complement Naive Bayes Model "action" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [148]:
cnb_tri_model_predictions = cnb_tri_model.predict(X_test)
cnb_tri_report = classification_report(y_test, cnb_tri_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Performance Report\n")
print(cnb_tri_report)
print()

Complement Naive Bayes Model "action" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        11

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11




In [149]:
print(cnb_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Word: claim

We pick two senses of claim. They are:
1.  S: (n) claim (claim%1:10:02::) (an assertion that something is true or factual) "his claim that he was innocent"; "evidence contradicted the government's claims"
2.  S: (n) claim (claim%1:04:00::) (demand for something as rightful or due) "they struck in support of their claim for a shorter work day"

We assign 1 to the first sense and 2 to the second sense.

In [150]:
# parameters to change
target_word = "claim"
sense_1_file = "/content/claim_assertion.txt"
sense_2_file = "/content/claim_demand.txt"
general_file = "/content/claim.txt"
sense_1 = "1:10:02::"
sense_2 = "1:04:00::"

In [151]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  9


In [152]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_uni_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_uni_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [153]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Best Hyperparameters\n")
print(cnb_uni_model.best_params_)
print()

Complement Naive Bayes Model "claim" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [154]:
cnb_uni_model_predictions = cnb_uni_model.predict(X_test)
cnb_uni_report = classification_report(y_test, cnb_uni_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Performance Report\n")
print(cnb_uni_report)
print()

Complement Naive Bayes Model "claim" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.89      0.94         9
           2       0.00      0.00      0.00         0

    accuracy                           0.89         9
   macro avg       0.50      0.44      0.47         9
weighted avg       1.00      0.89      0.94         9




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [155]:
print(cnb_uni_model_predictions)
print(y_test)

[1 1 1 1 1 1 2 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [156]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  9


In [157]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_bi_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_bi_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [158]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Best Hyperparameters\n")
print(cnb_bi_model.best_params_)
print()

Complement Naive Bayes Model "claim" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [159]:
cnb_bi_model_predictions = cnb_bi_model.predict(X_test)
cnb_bi_report = classification_report(y_test, cnb_bi_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Performance Report\n")
print(cnb_bi_report)
print()

Complement Naive Bayes Model "claim" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.89      0.94         9
           2       0.00      0.00      0.00         0

    accuracy                           0.89         9
   macro avg       0.50      0.44      0.47         9
weighted avg       1.00      0.89      0.94         9




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [160]:
print(cnb_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1 2 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [161]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  9


In [162]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_tri_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_tri_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [163]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Best Hyperparameters\n")
print(cnb_tri_model.best_params_)
print()

Complement Naive Bayes Model "claim" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [164]:
cnb_tri_model_predictions = cnb_tri_model.predict(X_test)
cnb_tri_report = classification_report(y_test, cnb_tri_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Performance Report\n")
print(cnb_tri_report)
print()

Complement Naive Bayes Model "claim" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9




In [165]:
print(cnb_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1 1 1 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]


### Word: time

We pick two senses of time. They are:
1.  S: (n) time (time%1:11:00::), clip (clip%1:11:00::) (an instance or single occasion for some event) "this time he succeeded"; "he called four times"; "he could do ten at a clip"
2. S: (n) time (time%1:28:05::) (a period of time considered as a resource under your control and sufficient to accomplish something) "take time to smell the roses"; "I didn't have time to finish"; "it took more than half my time"; "he waited for a long time"

We assign 1 to the first sense and 2 to the second sense.

In [166]:
# parameters to change
target_word = "time"
sense_1_file = "/content/time_event.txt"
sense_2_file = "/content/time_resource.txt"
general_file = "/content/time.txt"
sense_1 = "1:11:00::"
sense_2 = "1:28:05::"

In [167]:
# unigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "uni")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  6


In [168]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_uni_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_uni_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [169]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Best Hyperparameters\n")
print(cnb_uni_model.best_params_)
print()

Complement Naive Bayes Model "time" with Unigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [170]:
cnb_uni_model_predictions = cnb_uni_model.predict(X_test)
cnb_uni_report = classification_report(y_test, cnb_uni_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Unigram Proprocessing - Performance Report\n")
print(cnb_uni_report)
print()

Complement Naive Bayes Model "time" with Unigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       1.00      0.25      0.40         4
           2       0.40      1.00      0.57         2

    accuracy                           0.50         6
   macro avg       0.70      0.62      0.49         6
weighted avg       0.80      0.50      0.46         6




In [171]:
print(cnb_uni_model_predictions)
print(y_test)

[1 2 2 2 2 2]
[1, 1, 2, 2, 1, 1]


In [172]:
# bigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "bi")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  6


In [173]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_bi_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_bi_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [174]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Best Hyperparameters\n")
print(cnb_bi_model.best_params_)
print()

Complement Naive Bayes Model "time" with Bigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [175]:
cnb_bi_model_predictions = cnb_bi_model.predict(X_test)
cnb_bi_report = classification_report(y_test, cnb_bi_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Bigram Proprocessing - Performance Report\n")
print(cnb_bi_report)
print()

Complement Naive Bayes Model "time" with Bigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.67      1.00      0.80         4
           2       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [176]:
print(cnb_bi_model_predictions)
print(y_test)

[1 1 1 1 1 1]
[1, 1, 2, 2, 1, 1]


In [177]:
# trigram
X_train_semi, y_train_semi, X_train_labeled, y_train_labeled, X_test, y_test = output_train_test_sets(target_word, sense_1_file, sense_2_file, sense_1, sense_2, "tri")

Size of unlabeled training set:  100
Size of labeled training set:  101
Size of test set (labeled):  6


In [178]:
param_grid = {'alpha': [0, 1e-12, 1e-10, 1e-08] ,
              'class_prior': [[0.5, 0.5], None]}

cnb_tri_model = GridSearchCV(ComplementNB(force_alpha=True), param_grid, refit=True, verbose=3, n_jobs=-1)
cnb_tri_model.fit(X_train_labeled, y_train_labeled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(force_alpha=True), n_jobs=-1,
             param_grid={'alpha': [0, 1e-12, 1e-10, 1e-08],
                         'class_prior': [[0.5, 0.5], None]},
             verbose=3)

In [179]:
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Best Hyperparameters\n")
print(cnb_tri_model.best_params_)
print()

Complement Naive Bayes Model "time" with Trigram Proprocessing - Best Hyperparameters

{'alpha': 1e-12, 'class_prior': [0.5, 0.5]}



In [180]:
cnb_tri_model_predictions = cnb_tri_model.predict(X_test)
cnb_tri_report = classification_report(y_test, cnb_tri_model_predictions)
print(f"Complement Naive Bayes Model \"{target_word}\" with Trigram Proprocessing - Performance Report\n")
print(cnb_tri_report)
print()

Complement Naive Bayes Model "time" with Trigram Proprocessing - Performance Report

              precision    recall  f1-score   support

           1       0.67      1.00      0.80         4
           2       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [181]:
print(cnb_tri_model_predictions)
print(y_test)

[1 1 1 1 1 1]
[1, 1, 2, 2, 1, 1]


## Baseline model's performance on the five words

In [182]:
words = ["country", "game", "action", "claim", "time"]

In [183]:
# get the accuracies of the baseline model on the five specific words
for target_word in words:
    # go through all the test cases of a word
    test_size = 0
    accurate_assignments = 0
    for key in test_instances.keys():
        # get the lemma and the word sense
        word = test_instances[key].lemma.decode('utf-8').lower()
        if word == target_word:
            test_size += 1

            # get the sense key
            sense_key = wn.synsets(word)[0].lemmas()[0].key()
            sense_key = sense_key.split("%")[1]

            # get the gold standard
            for line in lines:
                if key in line:
                    gold_std = line.split(" ")[2]
                    gold_std = gold_std.split("%")[1]

            if sense_key == gold_std:
                accurate_assignments += 1
    accuracy = accurate_assignments / test_size
    print(f"Accuracy for baseline model on word \"{target_word}\": ", accuracy)

Accuracy for baseline model on word "country":  1.0
Accuracy for baseline model on word "game":  0.0
Accuracy for baseline model on word "action":  1.0
Accuracy for baseline model on word "claim":  0.1
Accuracy for baseline model on word "time":  0.6666666666666666


In [184]:
# print the prediction of the baseline model
for target_word in words:
    sense_key = wn.synsets(target_word)[0].lemmas()[0].key()
    sense_key = sense_key.split("%")[1]
    print(f"Baseline model's prediction for {target_word}: {sense_key}")

Baseline model's prediction for country: 1:14:00::
Baseline model's prediction for game: 1:04:00::
Baseline model's prediction for action: 1:04:02::
Baseline model's prediction for claim: 1:10:00::
Baseline model's prediction for time: 1:11:00::


## Lesk's model's performance on the five words

In [185]:
# get the accuracies of the Lesk's model on the five specific words
for target_word in words:
    # go through all the test cases and record the number of accurate word sense assignment
    test_size = 0
    accurate_assignments = 0
    for test_instance in test_processed:
        key = test_instance[0]

        # get the lemma and the word sense
        if test_instance[1] == target_word:
            test_size += 1

            # get the sense key
            word = test_instance[1]
            sense_key = lesk(test_instance[2], word, 'n').lemmas()[0].key()
            sense_key = sense_key.split("%")[1]

            # get the gold standard
            for gs in gold_standard:
                if key == gs[0]:
                    gold_std = gs[1]
                    gold_std = gold_std.split("%")[1]

            if sense_key == gold_std:
                accurate_assignments += 1
    accuracy = accurate_assignments / test_size
    print(f"Accuracy for Lesk's model on word \"{target_word}\": ", accuracy)

Accuracy for Lesk's model on word "country":  0.05555555555555555
Accuracy for Lesk's model on word "game":  0.0
Accuracy for Lesk's model on word "action":  0.0
Accuracy for Lesk's model on word "claim":  0.0
Accuracy for Lesk's model on word "time":  0.3333333333333333


In [186]:
# print the prediction for each test case
for target_word in words:
    # go through all the test cases and record the number of accurate word sense assignment
    print(f"Target word: {target_word}")
    print()
    test_size = 0
    accurate_assignments = 0
    for test_instance in test_processed:
        key = test_instance[0]

        # get the lemma and the word sense
        if test_instance[1] == target_word:
            test_size += 1
            print(f"\tTest case: {test_instance}")

            # get the sense key
            word = test_instance[1]
            sense_key = lesk(test_instance[2], word, 'n').lemmas()[0].key()
            sense_key = sense_key.split("%")[1]
            print(f"\tPredicted sense key: {sense_key}")

            # get the gold standard
            for gs in gold_standard:
                if key == gs[0]:
                    gold_std = gs[1]
                    gold_std = gold_std.split("%")[1]
                    print(f"\tGold standard sense key: {gold_std}")
                    print()
            if sense_key == gold_std:
                accurate_assignments += 1
    accuracy = accurate_assignments / test_size
    print()

Target word: country

	Test case: ('d009.s005.t004', 'country', ['finding', 'late', 'latinobarmetro', 'poll', 'take', '@card@', 'country', 'publish', 'exclusively', 'economist', '.'])
	Predicted sense key: 1:14:01::
	Gold standard sense key: 1:14:00::

	Test case: ('d009.s010.t003', 'country', ['support', 'democracy', 'rise', 'noticeably', 'several', 'country', 'pacific_rim', 'south_america', '(', 'see', 'table', '1', ')', '.'])
	Predicted sense key: 1:14:01::
	Gold standard sense key: 1:14:00::

	Test case: ('d009.s013.t006', 'country', ['around', 'half', 'respondent', 'mexico', 'brazil', ',', 'region', "'s", 'two', 'populous', 'country', ',', 'convinced', 'democrat', ',', 'drag', 'regional', 'average', '(', 'see', 'chart', '2', ')', '.'])
	Predicted sense key: 1:14:01::
	Gold standard sense key: 1:14:00::

	Test case: ('d009.s014.t004', 'country', ['@card@', '%', 'respondent', 'across', 'region', 'pronounce', 'satisfy', 'country', "'s", 'democracy', 'work', 'practice', ',', 'figure',

In [188]:
for target_word in words:
    for test_instance in test_processed:
        if test_instance[1] == target_word:
            id = test_instance[0]
            for id_key in test_instances.keys():
                if id_key == id:
                    print(str(test_instances[id_key]))

d009.s005.t004	b'country'	b'those be some of the finding of the late Latinobarmetro poll take in @card@ country and publish exclusively by the economist .'	14
d009.s010.t003	b'country'	b'support for democracy have rise noticeably in several country on the Pacific_rim of south_america ( see table 1 ) .'	8
d009.s013.t006	b'country'	b"but only around half of respondent in Mexico and Brazil , the region 's two most populous country , be convinced democrat , which drag down the regional average ( see chart 2 ) ."	17
d009.s014.t004	b'country'	b"some @card@ % of respondent across the region pronounce themselves satisfy with how their country 's democracy work in practice , the same figure as last year but a big increase on the @card@ % in @card@ ( chart 3 ) ."	14
d009.s015.t003	b'country'	b"some @card@ % say they have confidence in their country 's Congress and @card@ % in their government , the same as last year in both case but up sharply from @card@ ( @card@ % and @card@ % respectively ) .